In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

print(torch.cuda.is_available())


True


In [3]:
device = torch.device('cuda')
batch_size = 64
num_classes = 10
learning_rate = 0.001
epochs = 10

In [4]:
from torchvision import datasets

#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                               train = True,
                                               transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                               download = True)
    
    
test_dataset = torchvision.datasets.MNIST(root = './data',
                                              train = False,
                                              transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                              download=True)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [11]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        
        self.layer1 = nn.Sequential(
                nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(6),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
                nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, 10)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
        

In [12]:
model = LeNet5().to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)

In [13]:
for e in range(epochs):
    for i , (images, labels) in enumerate(train_loader):
        images = images.to(device)    
        labels = labels.to(device)
        
        outputs = model(images)
        loss = loss_function(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(e+1, epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/938], Loss: 0.1837
Epoch [1/10], Step [200/938], Loss: 0.2353
Epoch [1/10], Step [300/938], Loss: 0.0597
Epoch [1/10], Step [400/938], Loss: 0.1090
Epoch [1/10], Step [500/938], Loss: 0.1516
Epoch [1/10], Step [600/938], Loss: 0.0459
Epoch [1/10], Step [700/938], Loss: 0.0436
Epoch [1/10], Step [800/938], Loss: 0.0650
Epoch [1/10], Step [900/938], Loss: 0.0056
Epoch [2/10], Step [100/938], Loss: 0.0868
Epoch [2/10], Step [200/938], Loss: 0.1154
Epoch [2/10], Step [300/938], Loss: 0.0800
Epoch [2/10], Step [400/938], Loss: 0.0837
Epoch [2/10], Step [500/938], Loss: 0.0726
Epoch [2/10], Step [600/938], Loss: 0.0610
Epoch [2/10], Step [700/938], Loss: 0.0419
Epoch [2/10], Step [800/938], Loss: 0.0263
Epoch [2/10], Step [900/938], Loss: 0.0123
Epoch [3/10], Step [100/938], Loss: 0.0209
Epoch [3/10], Step [200/938], Loss: 0.0620
Epoch [3/10], Step [300/938], Loss: 0.1015
Epoch [3/10], Step [400/938], Loss: 0.0303
Epoch [3/10], Step [500/938], Loss: 0.0030
Epoch [3/10

In [16]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))



Accuracy of the network on the 10000 test images: 99.19 %
